In [1]:
from splinter import Browser
from splinter.exceptions import ElementDoesNotExist
from selenium.common.exceptions import StaleElementReferenceException
from bs4 import BeautifulSoup as bs
import pymongo
import time

In [3]:
# mongodb connection
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

db = client.job_description_db2

#db.jd.delete_many({}) # clean collection

In [4]:
cities = ['San Diego, CA', 'New York, NY', 'Los Angeles, CA', 'Chicago, IL', 'Washington, DC', 'San Francisco, CA',
          'Boston, MA', 'Dallas, TX', 'Philadelphia, PA', 'Houston, TX', 'Miami, FL', 'Atlanta, GA',
          'Detroit, MI', 'Seattle, WA', 'Minneapolis, MN', 'Cleveland, OH', 'Denver, CO', 'Orlando, FL', 'Portland, OR',
          'St. Louis, MO', 'Pittsburgh, PA', 'Charlotte, NC', 'Sacramento, CA', 'Salt Lake City, UT', 'Kansas City, MO',
          'Columbus, OH', 'Las Vegas, NV', 'Indianapolis, IN', 'Cincinnati, OH', 'Raleigh, NC', 'Milwaukee, WI' ]

job_queries = ['"Data Scientist"']

In [5]:
# init browser
browser = Browser('chrome')
browser.visit('http://glassdoor.com')

In [13]:
_ = input("Note: you may need to login in glassdoor using the chrome driver window before proceeding.\n"\
          "Hit <Enter> when you are ready.")

for city in cities:
    for job_title_query in job_queries:
        
        # query glassdoor
        browser.find_by_id('sc.keyword').fill(job_title_query)
        browser.find_by_id('sc.location').fill(city)
        browser.click_link_by_id('HeroSearchButton')
        
        # Use the browser to find the <li> tags, then use soup to scrape
        # We need the browser for clicking on the <li> and get the detailed job description
        job_list = browser.find_by_id('MainCol')[0].find_by_tag('ul')[0].find_by_tag('li')
        
        while len(job_list) > 0:

            # extract job title, employer, salary and detailed description
            for li in job_list:
                
                # click on job hit
                li.click()
                
                time.sleep(0.5)
                
                soup_li = bs(li.html, 'html.parser')
                
                try:
                    job_title = soup_li.find('div', class_='jobTitle').find('div').text.strip()
                except:
                    job_title = 'ERROR'
                    
                try:
                    employer_div = soup_li.find('div', class_='empLoc').find('div')
                    job_employer = ''.join(employer_div.text.split('–')[0:-1]).strip() # remove the city name
                    job_employer_city = employer_div.find('span').text.strip()
                    if job_employer.startswith('Indeed Prime'): continue
                except:
                    job_employer = 'ERROR'
                
                try:
                    job_salary = soup_li.find('span', class_='green').text.strip()
                except:
                    job_salary = "ERROR"
                
                try:
                    job_description = browser.find_by_id('JobDescriptionContainer')[0].value
                except StaleElementReferenceException:
                    time.sleep(2)
                    # try again
                    job_description = browser.find_by_id('JobDescriptionContainer')[0].value
                except ElementDoesNotExist:
                    continue
                    
                
                #print(job_title + ' ' + job_employer + ' ' + job_employer_city + ' '+ job_salary + ' ' + job_description[0:100] )
                
                post = {'query_title':job_title_query, "query_city":city, 'job_title':job_title, \
                        'job_employer':job_employer,'job_employer_city':job_employer_city, \
                        'job_salary':job_salary, 'job_decription':job_description}
                x = db.jd.insert_one(post)                
            
            # click on next button
            try:
                li_next = browser.find_by_id('FooterPageNav')[0].find_by_tag('li').last
            except ElementDoesNotExist:
                job_list = []
                continue
            if li_next.find_by_tag('span')[0].has_class('disabled'):
                # reached end of list
                job_list = []
            else:
                # get next list
                li_next.click()
                job_list = browser.find_by_id('MainCol')[0].find_by_tag('ul')[0].find_by_tag('li')
                

Note: you may need to login in glassdoor using the chrome driver window before proceeding.
Hit <Enter> when you are ready.
